# Run OpenMM on the HPC


**<span style="color:#A03;font-size:14pt">
&#x270B; HANDS-ON! &#x1F528;
</span>**

- Copy this notebook and the file alanine-dipeptide.pdb to a directory on the HPC.
- Create a job file `job_openmm.sh` to run the notebook

```
#!/usr/bin/env bash
#PBS -N _openmm
#PBS -l nodes=1:ppn=12


# Load the OpenMM module, still needs to be installed...

# Change the filename of the notebook below
cd ${PBS_O_WORKDIR}
export OPENMM_NUM_THREADS=${SLURM_CPUS_ON_NODE}
time jupyter nbconvert --to notebook --execute --allow-errors --ExecutePreprocessor.timeout=-1 mynotebook.ipynb
```

- After preparing these files, run `qsub job_openmm.sh`.

In [ ]:
# Import all the modules we need.
%matplotlib widget
from sys import stdout
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *

In [ ]:
# Copy the code from notebook2, example 3
pdb = PDBFile('alanine-dipeptide.pdb')
modeller = Modeller(pdb.topology, pdb.positions)
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
modeller.addSolvent(forcefield, model='tip3p', padding=1*nanometer)
print(modeller.topology)
# Write a PDB file to provide a topology of the solvated
# system to MDTraj below.
with open('init3.pdb', 'w') as outfile:
    PDBFile.writeFile(modeller.topology, modeller.positions, outfile)

# The modeller builds a periodic box with the solute and solvent molecules.
# PME is the method to compute long-range electristatic interactions in
# periodic systems.
system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME, constraints=HBonds)
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 2*femtoseconds)
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)
simulation.minimizeEnergy()
simulation.reporters.append(DCDReporter('traj3.dcd', 100))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
        temperature=True, elapsedTime=True))
simulation.reporters.append(StateDataReporter("scalars3.csv", 100, time=True,
    potentialEnergy=True, totalEnergy=True, temperature=True))
simulation.step(100000)